This notebook is part of the *orix* documentation https://orix.rtfd.io. Links to the documentation won’t work from the notebook.

# Clustering across fundamental region boundaries

This tutorial demonstrates density based clustering of crystal orientations with
and without the application of crystal symmetry using simulated data.

Import orix classes and various dependencies

In [ ]:
# exchange inline for notebook (or qt5 from pyqt) for interactive plotting
%matplotlib inline

# Import core external
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Colorisation & Animation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb
import matplotlib.animation as animation

# Import orix classes 
from orix import plot
from orix.quaternion import Orientation, OrientationRegion, Rotation
from orix.quaternion.symmetry import C1, Oh
from orix.vector import AxAngle


plt.rcParams.update({"figure.figsize": (10, 5)})

## Generate artificial data

Generate three random von Mises distributions of orientations as model clusters
and set the *Oh* ($m\bar{3}m$) point group symmetry

In [ ]:
n_orientations = 50
alpha = 50  # Lower value lead to "looser" distribution

# Cluster 1
cluster1 = Rotation.random_vonmises(n_orientations, alpha=alpha)

# Cluster 2
centre2 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.pi / 4))
cluster2 = Rotation.random_vonmises(n_orientations, alpha=alpha, reference=centre2)

# Cluster 3
centre3 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 1, 0), np.pi / 3))
cluster3 = Rotation.random_vonmises(n_orientations, alpha=alpha, reference=centre3)

ori = Orientation.stack([cluster1, cluster2, cluster3]).flatten().set_symmetry(Oh)

## Orientation clustering

### Perform clustering without application of crystal symmetry

Compute misorientations, i.e. distance between orientations

In [ ]:
# Remove symmetry by setting it to point group 1 (identity operation)
ori_without_symmetry = ori.set_symmetry(C1)

# Misorientations
mori1 = (~ori_without_symmetry).outer(ori_without_symmetry)

# Misorientation angles
D1 = mori1.angle.data

Perform clustering

In [ ]:
dbscan_naive = DBSCAN(eps=0.3, min_samples=10, metric="precomputed").fit(D1)
print("Labels:", np.unique(dbscan_naive.labels_))

### Perform clustering with application of crystal symmetry

Compute misorientations, i.e. distance between orientations, with symmetry

In [ ]:
mori2 = (~ori).outer(ori).set_symmetry(Oh)
D2 = mori2.angle.data

# Or get the angles in one call
#D2 = ori.get_distance_matrix(ori).data

Perform clustering

In [ ]:
dbscan = DBSCAN(eps=0.3, min_samples=20, metric="precomputed").fit(D2)
print("Labels:", np.unique(dbscan.labels_))

This should have shown that without symmetry there are 6 clusters, whereas with symmetry there are 3.

## Visualisation

Assign colours to each cluster

In [ ]:
color_names = [to_rgb(f"C{i}") for i in range(6)]  # ['C0', 'C1', ...]
colors = label2rgb(dbscan.labels_, colors=color_names, bg_label=-1)
colors_naive = label2rgb(dbscan_naive.labels_, colors=color_names, bg_label=-1)

Specify fundamental zone based on symmetry

In [ ]:
fr = OrientationRegion.from_symmetry(ori.symmetry)

Generate plot

In [ ]:
fig, ax = plt.subplots(ncols=2, subplot_kw=dict(projection="axangle"))
wireframe_kwds = dict(color="gray", alpha=0.5, linewidth=0.5, rcount=30, ccount=30)
box_aspects = (1,) * 3

# Clustering without symmetry
ax[0].scatter(ori_without_symmetry, c=colors_naive)
ax[0].plot_wireframe(fr, **wireframe_kwds)
ax[0].axis("off")
ax[0].set_box_aspect(box_aspects)

# Clustering with symmetry
ax[1].scatter(ori, c=colors)
ax[1].plot_wireframe(fr, **wireframe_kwds)
ax[1].axis("off")
ax[1].set_box_aspect(box_aspects)

fig.tight_layout(w_pad=-5)

Generate an animation of the plot (assuming an interactive Matplotlib backend is
used)

In [ ]:
def animate(angle):
    ax[0].view_init(15, angle)
    ax[1].view_init(15, angle)
    plt.draw()

ani = animation.FuncAnimation(fig, animate, np.linspace(75, 360+74, 720), interval=25)

(Note that you might have to move the figure window on your screen for the
animation to start.)